In [ ]:
!pip install google-genai pydantic

In [2]:
import os
os.environ['GEMINI_API_KEY'] = 'MY_API_KEY'

In [4]:
import os
import json
from google import genai
from pydantic import BaseModel, Field

# Initialize the Google GenAI client (uses GEMINI_API_KEY from environment)
client = genai.Client()
MODEL_ID = 'gemini-2.5-flash'

# --- 1. Define External Tools (The Environment) ---

def get_current_position() -> str:
    """Tool: Returns the robot's current coordinates."""
    return '{"x": 0, "y": 0}'

def get_goal_position() -> str:
    """Tool: Returns the target destination coordinates."""
    return '{"x": 5, "y": 5}'

def move_robot(x: int, y: int) -> str:
    """Tool: Moves the robot to the specified coordinates."""
    return f"Success: Robot moved to coordinates ({x}, {y})."

AVAILABLE_TOOLS = {
    "get_current_position": get_current_position,
    "get_goal_position": get_goal_position,
    "move_robot": move_robot
}

# --- 2. Define the Structured Schema for ReAct ---

class ReActDecision(BaseModel):
    thought: str = Field(description="Your reasoning about the current state, what you've observed, and what to do next.")
    action: str = Field(description="The exact name of the tool to use (e.g., 'get_current_position', 'move_robot') OR 'FINISH' if the task is complete.")
    # FIX: Changed from dict to a stringified JSON to bypass the additionalProperties restriction.
    action_input_json: str = Field(description="A strict JSON string of the tool arguments. Example: '{\"x\": 5, \"y\": 5}'. Use '{}' if no arguments are needed. If action is FINISH, put the final answer here.")

# --- 3. The ReAct Loop (Orchestrator) ---

def react_agent(user_prompt: str, max_iterations: int = 5) -> str:
    print(f"Starting ReAct Agent for task: '{user_prompt}'\n")

    memory_context = f"Task: {user_prompt}\n\nAvailable Tools: {list(AVAILABLE_TOOLS.keys())}\n"
    iteration = 1

    while iteration <= max_iterations:
        print(f"--- Iteration {iteration} ---")

        prompt = f"""You are a ReAct agent. Read the memory and decide your next step.
        {memory_context}
        """

        response = client.models.generate_content(
            model=MODEL_ID,
            contents=prompt,
            config={
                'response_mime_type': 'application/json',
                'response_schema': ReActDecision,
                'temperature': 0.1,
            },
        )
        decision = response.parsed

        # Parse the JSON string into a Python dictionary
        parsed_args = {}
        if decision.action.upper() != "FINISH":
            try:
                parsed_args = json.loads(decision.action_input_json)
            except json.JSONDecodeError:
                print(f"⚠️ [Warning]: Model provided invalid JSON for arguments: {decision.action_input_json}")

        print(f"🤔 [Thought]: {decision.thought}")
        print(f"🛠️  [Action]: {decision.action} with input {decision.action_input_json}")

        # Append to memory
        memory_context += f"\nThought: {decision.thought}\nAction: {decision.action}\nAction Input: {decision.action_input_json}\n"

        # Check Exit Condition
        if decision.action.upper() == "FINISH":
            print("\n✅ [End Condition Met]: Task is complete.")
            return decision.action_input_json

        # Execute Tool
        observation = ""
        if decision.action in AVAILABLE_TOOLS:
            tool_func = AVAILABLE_TOOLS[decision.action]
            try:
                # Unpack the safely parsed dictionary arguments
                observation = tool_func(**parsed_args)
            except Exception as e:
                observation = f"Error executing tool: {e}"
        else:
            observation = f"Error: Tool '{decision.action}' not found."

        print(f"👀 [Observation]: {observation}\n")
        memory_context += f"Observation: {observation}\n"
        iteration += 1

    print("\n⚠️ [Max Iterations Reached]: Terminating loop to prevent infinite execution.")
    return "Task failed to complete within the iteration limit."

# --- Execution ---
if __name__ == "__main__":
    task = "Find my current position, find the goal position, and then move the robot to the goal."
    final_output = react_agent(task)
    print("\n=== Final Output ===")
    print(final_output)

Starting ReAct Agent for task: 'Find my current position, find the goal position, and then move the robot to the goal.'

--- Iteration 1 ---
🤔 [Thought]: The task requires me to first find the current position, then the goal position, and finally move the robot to the goal. I should start by getting the current position of the robot.
🛠️  [Action]: get_current_position with input {}
👀 [Observation]: {"x": 0, "y": 0}

--- Iteration 2 ---
🤔 [Thought]: I have already obtained the current position. The next step is to get the goal position.
🛠️  [Action]: get_goal_position with input {}
👀 [Observation]: {"x": 5, "y": 5}

--- Iteration 3 ---
🤔 [Thought]: I have successfully obtained both the current position and the goal position. The next step is to move the robot to the goal position, which is (5,5). I will use the 'move_robot' tool for this.
🛠️  [Action]: move_robot with input {"x": 5, "y": 5}
👀 [Observation]: Success: Robot moved to coordinates (5, 5).

--- Iteration 4 ---
🤔 [Thought]: I 